# Differential Privacy를 이용한 연합 학습

#### 1. Load imports

In [10]:
import sys

print(sys.executable)

/home/ec2-user/anaconda3/envs/python3/bin/python


In [11]:
print(sys.version)

3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:19:12) [GCC 13.3.0]


In [12]:
print(sys.version)

3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:19:12) [GCC 13.3.0]


In [13]:
!{sys.executable} -m pip install -r requirements.txt

필요한 패키지를 import 합니다.

In [1]:
from flwr.client.mod import adaptiveclipping_mod
from flwr.server.strategy import (
    DifferentialPrivacyClientSideAdaptiveClipping,
    FedAvg,
)

from utils4 import *

2025-11-26 02:18:18.914691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764123499.120568    7984 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764123499.184139    7984 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764123499.713049    7984 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764123499.713077    7984 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764123499.713079    7984 computation_placer.cc:177] computation placer alr

#### 2. Load the MNIST dataset

* 연합 학습을 경험할 수 있는 `flwr-datasets` 데이터셋을 이용합니다.

In [2]:
def load_data(partition_id):
    fds = FederatedDataset(dataset="mnist", partitioners={"train": 10})
    partition = fds.load_partition(partition_id)

    traintest = partition.train_test_split(test_size=0.2, seed=42)
    traintest = traintest.with_transform(normalize)
    trainset, testset = traintest["train"], traintest["test"]

    trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
    testloader = DataLoader(testset, batch_size=64)
    return trainloader, testloader

#### 3. Define the Model

* FlowerClient를 정의합니다.

In [3]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def fit(self, parameters, config):
        set_weights(self.net, parameters)
        train_model(self.net, self.trainloader)
        return get_weights(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_weights(self.net, parameters)
        loss, accuracy = evaluate_model(self.net, self.testloader)
        return loss, len(self.testloader), {"accuracy": accuracy}


def client_fn(context: Context) -> Client:
    net = SimpleModel()
    partition_id = int(context.node_config["partition-id"])
    trainloader, testloader = load_data(partition_id=partition_id)
    return FlowerClient(net, trainloader, testloader).to_client()

* ClientApp을 정의합니다.

특히, `mods=[adaptiveclipping_mod]` 는 아래와 같은 역할을 수행합니다.

1. 클라이언트의 gradient 또는 weight update를 받아
2. norm을 측정한 뒤
3. gradient clipping threshold(C)를 동적으로(adaptive) 조절하고
4. clip된 업데이트만 서버로 반환

In [4]:
client = ClientApp(
    client_fn,
    mods=[adaptiveclipping_mod],  # modifiers
)

* 서버에서도 Server DP에 대한 전략이 적용되도록 `FedAvg` 부분을 수정합니다.

**DP:** Differential Privacy.

In [5]:
net = SimpleModel()
params = ndarrays_to_parameters(get_weights(net))

def server_fn(context: Context):
    fedavg_without_dp = FedAvg(
        fraction_fit=0.6,
        fraction_evaluate=1.0,
        initial_parameters=params,
    )
    fedavg_with_dp = DifferentialPrivacyClientSideAdaptiveClipping(
        fedavg_without_dp,  # <- wrap the FedAvg strategy
        noise_multiplier=0.3,
        num_sampled_clients=6,
    )
    
    # Adjust to 50 rounds to ensure DP guarantees hold
    # with respect to the desired privacy budget
    config = ServerConfig(num_rounds=5)
    
    return ServerAppComponents(
        strategy=fedavg_with_dp,
        config=config,
    )

In [6]:
server = ServerApp(server_fn=server_fn)

* 아래 구문을 이용해서 연합 학습을 수행합니다.

In [7]:
run_simulation(server_app=server,
               client_app=client,
               num_supernodes=10,
               backend_config=backend_setup
               )

INFO : Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO : 
INFO : [INIT]
INFO : Using initial global parameters provided by strategy
INFO : Evaluating initial global parameters
INFO : 
INFO : [ROUND 1]
INFO : configure_fit: strategy sampled 6 clients (out of 10)
(raylet) /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/ray/_private/parameter.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(raylet)   import pkg_resources
(raylet) /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/ray/_private/parameter.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setu